# Plot GHRSST night time scenes

Use data on IMOS s3 bucket

http://imos-data.s3-website-ap-southeast-2.amazonaws.com/?prefix=IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/

In [7]:
import os, glob
import xarray as xr
import s3fs
import imageio

import matplotlib.pyplot as plt
import numpy as np
import cmocean  # Oceanography colour maps


In [8]:
#from utils import plot_swot_basemap
def plot_swot_basemap(ax, xlim, ylim):
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    ax.set_extent([xlim[0], xlim[1], ylim[0], ylim[1]], crs=ccrs.PlateCarree())
    ax.coastlines(resolution="10m", color="black", linewidth=0.8)
    ax.add_feature(cfeature.LAND, facecolor="lightgray")
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False
    return ax


In [9]:
plt.rcParams['font.size']=14
plt.rcParams['axes.labelsize']='medium'

In [10]:
# Login to the s3 bucket and list the folders in the base directory of the imos-data/ bucket

fs = s3fs.S3FileSystem(anon=True) # Note that we can login anonymously because this bucket allows it

aodn = fs.ls('imos-data/')
for item in aodn:
    print(item)

imos-data/AIMS
imos-data/Bureau_of_Meteorology
imos-data/CSIRO
imos-data/Charles_Darwin_University
imos-data/Curtin_University
imos-data/Deakin_University
imos-data/Defence_Technology_Agency-New_Zealand
imos-data/Department_for_Environment_and_Water-South_Australia
imos-data/Department_of_Defence
imos-data/Department_of_Environment_and_Science-Queensland
imos-data/Department_of_Planning_and_Environment-New_South_Wales
imos-data/Department_of_Transport-Western_Australia
imos-data/Derwent_Estuary_Program
imos-data/Flinders_University
imos-data/Future_Reef_MAP
imos-data/Gippsland-Ports-Victoria
imos-data/IMOS
imos-data/Macquarie_University
imos-data/NSW-OEH
imos-data/Pilbara_Ports_Authority
imos-data/Southern_Cross_University
imos-data/TERN
imos-data/UNSW
imos-data/UON
imos-data/UWA
imos-data/WAMSI
imos-data/aodn.js
imos-data/error.html
imos-data/index.html
imos-data/robots.txt


In [11]:
timestr = '202508'
aodn = fs.glob(f'imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/{timestr}*.nc')

for item in aodn:
    print(item)



imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250801152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250802152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250803152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250804152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250805152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250806152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250807152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250808152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2025/20250809152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_n

In [12]:
def download_netcdf(files_on_s3, outdir="./DATA/GHRSST"):
    os.makedirs(outdir, exist_ok=True)
    local_files = []
    for key in files_on_s3:
        fname = os.path.basename(key)
        local_path = os.path.join(outdir, fname)
        if not os.path.exists(local_path):
            print(f"Downloading {key} -> {local_path}")
            fs.get(key, local_path)    # s3fs 下载
        else:
            print(f"Skip existing: {local_path}")
        local_files.append(local_path)
    return sorted(local_files)

local_nc_files = download_netcdf(aodn, outdir="./DATA/GHRSST")
print(f"local files: {len(local_nc_files)}")


local files: 11


In [13]:
import os
import cartopy.crs as ccrs

def open_ds_any(ncfile: str):
    """本地路径用 xr.open_dataset；S3 键用 fs.open"""
    if ncfile.startswith("imos-data/") or ncfile.startswith("s3://"):
        with fs.open(ncfile) as f:
            return xr.open_dataset(f)
    else:
        return xr.open_dataset(ncfile)

def plot_scene(ncfile, timestr):
    print("Plot:", timestr, "|", ncfile)
    sst = open_ds_any(ncfile)

    # 裁剪经纬度范围（按需要改）
    x1, x2 = 111, 114
    y1, y2 = -15.5, -12
    mysst = sst.sel(lon=slice(x1, x2), lat=slice(y2, y1))

    # SST 变量名兜底
    for v in ["sea_surface_temperature", "analysed_sst", "sst"]:
        if v in mysst.variables:
            varname = v
            break
    else:
        raise KeyError(f"No SST variable found in {ncfile}")

    da = mysst[varname].squeeze()

    # 防御：像元至少 2x2 才能算梯度
    if (da.sizes.get("lat", 0) < 2) or (da.sizes.get("lon", 0) < 2):
        print(f"Skip (too small for gradient): {da.sizes}")
        return

    dx, dy = np.gradient(da.values)
    x = mysst['lon'].values
    y = mysst['lat'].values

    fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})

    da.plot(
        ax=ax,
        cmap=cmocean.cm.thermal,
        vmin=26+273.15, vmax=29+273.15,
        cbar_kwargs={"shrink": 0.3},
        transform=ccrs.PlateCarree()
    )

    ax.pcolormesh(
        x, y, np.abs(dx + 1j*dy),
        vmin=0, vmax=0.1, cmap='bone_r', alpha=0.5,
        transform=ccrs.PlateCarree()
    )

    plot_swot_basemap(ax, (x1, x2), (y1, y2))

    os.makedirs('../FIGURES', exist_ok=True)
    out_png = f'../FIGURES/GHRSST_1km_{timestr}.png'
    plt.tight_layout()
    plt.savefig(out_png, dpi=150)
    plt.close(fig)
    print("Saved:", out_png)




In [14]:
# 如果继续用 S3 打开，保持你原来的 for aodn...
# 如果用本地文件打开，改成：
for ncfile in local_nc_files:
    timestr = os.path.basename(ncfile)[0:8]
    # 本地打开：xr.open_dataset(ncfile) 即可
    # 可以把 plot_scene 改成接受“本地/远程通用”，比如检测前缀：
    plot_scene(ncfile, timestr)


Plot: 20250801 | ./DATA/GHRSST\20250801152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
Saved: ../FIGURES/GHRSST_1km_20250801.png
Plot: 20250802 | ./DATA/GHRSST\20250802152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
Saved: ../FIGURES/GHRSST_1km_20250802.png
Plot: 20250803 | ./DATA/GHRSST\20250803152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
Saved: ../FIGURES/GHRSST_1km_20250803.png
Plot: 20250804 | ./DATA/GHRSST\20250804152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
Saved: ../FIGURES/GHRSST_1km_20250804.png
Plot: 20250805 | ./DATA/GHRSST\20250805152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
Saved: ../FIGURES/GHRSST_1km_20250805.png
Plot: 20250806 | ./DATA/GHRSST\20250806152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
Saved: ../FIGURES/GHRSST_1km_20250806.png
Plot: 20250807 | ./DATA/GHRSST\20250807152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
Saved: ../FIGURES/GHRSST_1km_20250807.png
Plot: 20250808 | ./DATA/GHRSST\20250808152000-ABOM-L3C_GHRSST-

In [15]:
import imageio
import glob
import os

# 找到刚才生成的所有 PNG（按时间排序）
png_files = sorted(glob.glob("../FIGURES/GHRSST_1km_*.png"))

# 读取并生成 GIF
images = [imageio.v2.imread(f) for f in png_files]
gif_path = "../FIGURES/GHRSST_1km_anim.gif"

# duration 是每帧的时间（秒），你可以改成 0.5 / 1.0 等
imageio.mimsave(gif_path, images, duration=0.5)

print(f"GIF 已保存到 {gif_path}")


GIF 已保存到 ../FIGURES/GHRSST_1km_anim.gif
